In [ ]:
import numpy as np
import json, requests
import pandas as pd
import random
import cmlapi
import os

### Overview

the notebook will:
1. take input format for python endpoint
2. extract model url and model access key
3. reading input data, will take a single sample of link/location combination 
4. then take a random sample of 25 contiguous rows
5. Convert this into json format through 'dataframe_to_json' function
6. Put all pieces together for json payload required by endpoint
7. Send prediction request
8. 'Unpack' response

In [ ]:
window = 24

In [ ]:
def dataframe_to_json(df):
    """Create a dictionary that will later be converted to a JSON object
    ensure that 
    """
    data = {
        'rx_gbs': df['rx_gbs'].iloc[-(window+1):].tolist(),
        'tx_gbs': df['tx_gbs'].iloc[-(window+1):].tolist(),
        'time': [df['time'].iloc[-1]],  # Only the last time entry
        'link': [df['link'].iloc[-1]],  # Only the last link entry
        'location': [df['location'].iloc[-1]]  # Only the last location entry
    }
    
    # build embedded dictionary step 1
    request_dict = {"request":data}

    # access key will be end point specific
    BackDict = {"accessKey":model_key}
    BackDict.update(request_dict)
    request_dict=BackDict
    
    return request_dict

In [ ]:
df = pd.read_csv('data/netstats_hourly_4_3.csv')
#df.head()

In [ ]:
model_name = input("Enter the name of the model: ")

In [ ]:
## Need to get the below prgramatically in the future iterations
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
target_model = client.list_all_models(search_filter=json.dumps({"name": model_name}))
model_key = target_model.models[0].access_key
model_url = os.getenv("CDSW_API_URL").replace("https://", "https://modelservice.").replace("/api/v1", "/model")

In [ ]:
target_model.models[0].crn

Pick a random link / location

In [ ]:
sample_df = df[['link','location']].drop_duplicates().sample()
link_pick = sample_df['link'].values[0]
location_pick = sample_df['location'].values[0]
print(sample_df)

In [ ]:
# test_df has only one combintation  of link_pick and location_pick
test_df = df[(df.link == link_pick) & (df.location == location_pick)]

In [ ]:
# pick a random segment of size window + 1
start_row = np.random.randint(0, len(test_df) - (window+1))
random_segment = df.iloc[start_row:start_row + (window+1)]

In [ ]:
# confirm size
random_segment.shape

In [ ]:
random_segment.head()

In [ ]:
# convert input to json
request_dict = dataframe_to_json(random_segment)

In [ ]:
#test_inputs

Step below puts it into required payload for endpoint

In [ ]:
# # build embedded dictionary step 1
# request_dict = {"request":test_inputs}

# # access key will be end point specific
# BackDict = {"accessKey":model_key}
# BackDict.update(request_dict)
# request_dict=BackDict

In [ ]:
r = requests.post(model_url, data=json.dumps(request_dict), headers={'Content-Type': 'application/json'})

In [ ]:
r.json()

Get the model response

In [ ]:
r.json()['response']['prediction']['time']

In [ ]:
r.json()['response']['prediction']['rx_bytes']

In [ ]:
r.json()['response']['prediction']['tx_bytes']

In [ ]:
r.json()['response']['uuid']

In [ ]:
type(rx_response)

In [ ]:
type(rx_response[0])